In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-extraction/20211030 Test Assignment/Input.xlsx
/kaggle/input/data-extraction/20211030 Test Assignment/Output Data Structure.xlsx
/kaggle/input/data-extraction/20211030 Test Assignment/Objective.docx
/kaggle/input/data-extraction/20211030 Test Assignment/Text Analysis.docx
/kaggle/input/data-extraction/20211030 Test Assignment/MasterDictionary/negative-words.txt
/kaggle/input/data-extraction/20211030 Test Assignment/MasterDictionary/positive-words.txt
/kaggle/input/data-extraction/20211030 Test Assignment/StopWords/StopWords_Currencies.txt
/kaggle/input/data-extraction/20211030 Test Assignment/StopWords/StopWords_GenericLong.txt
/kaggle/input/data-extraction/20211030 Test Assignment/StopWords/StopWords_Names.txt
/kaggle/input/data-extraction/20211030 Test Assignment/StopWords/StopWords_Generic.txt
/kaggle/input/data-extraction/20211030 Test Assignment/StopWords/StopWords_DatesandNumbers.txt
/kaggle/input/data-extraction/20211030 Test Assignment/StopWords/StopWords_Geo

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import re
import nltk
from nltk.corpus import stopwords
from textblob import Word

In [3]:
# Function to extract article text from URL
def extract_text(url):
    # Use requests to get the HTML content of the page
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract article title
    article_title = soup.find('title').get_text()
    
    # Extract article text
    article_text = ''
    for paragraph in soup.find_all('p'):
        article_text += paragraph.get_text() + '\n'
    
    return article_title, article_text


In [4]:
from nltk.corpus import cmudict

In [5]:
# Initialize CMU Pronouncing Dictionary
cmu_dict = cmudict.dict()

In [6]:
 #Function to count syllables in a word
def syllable_count(word):
    if word.lower() in cmu_dict:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in cmu_dict[word.lower()]])
    else:
        # Simple fallback: count vowels
        return max(1, sum(1 for letter in word.lower() if letter in 'aeiouy'))


In [7]:
# Function to compute text analysis variables
def compute_variables(text, positive_words, negative_words):
    # Use TextBlob for sentiment analysis
    blob = TextBlob(text)
    
    # Compute sentiment scores
    positive_score = sum(1 for word in blob.words if word.lower() in positive_words)
    negative_score = sum(1 for word in blob.words if word.lower() in negative_words)
    polarity_score = (positive_score - negative_score) / max(positive_score, negative_score, 1)
    subjectivity_score = blob.sentiment.subjectivity
    
    # Compute other variables
    sentences = blob.sentences
    avg_sentence_length = sum(len(sentence.words) for sentence in sentences) / len(sentences)
    words = [word for sentence in sentences for word in sentence.words]
    complex_words = [word for word in words if syllable_count(word) > 2]
    percentage_complex_words = (len(complex_words) / len(words)) * 100
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    avg_words_per_sentence = len(words) / len(sentences)
    complex_word_count = len(complex_words)
    word_count = len(words)
    syllable_per_word = sum(syllable_count(word) for word in words) / len(words)
    personal_pronouns = sum(1 for word in words if word.lower() in ['i', 'me', 'my', 'mine', 'myself', 'we', 'us', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves'])
    avg_word_length = sum(len(word) for word in words) / len(words)
    
    return positive_score, negative_score, polarity_score, subjectivity_score, \
           avg_sentence_length, percentage_complex_words, fog_index, \
           avg_words_per_sentence, complex_word_count, word_count, \
           syllable_per_word, personal_pronouns, avg_word_length

In [8]:
# Load positive and negative words from files
with open('/kaggle/input/data-extraction/20211030 Test Assignment/MasterDictionary/positive-words.txt', encoding = "ISO-8859-1") as f:
    positive_words = set(f.read().splitlines())
with open('/kaggle/input/data-extraction/20211030 Test Assignment/MasterDictionary/negative-words.txt', encoding = "ISO-8859-1") as f:
    negative_words = set(f.read().splitlines())

# Read input Excel file
input_data = pd.read_excel('/kaggle/input/data-extraction/20211030 Test Assignment/Input.xlsx')

# Initialize list to store output data
output_data = []

In [9]:
# Process each URL in the input data
for index, row in input_data.iterrows():
    url = row['URL']
    url_id = row['URL_ID']
    
    # Extract article text
    article_title, article_text = extract_text(url)
    
    # Compute variables
    variables = compute_variables(article_text, positive_words, negative_words)
    
    # Prepare output row
    output_row = [url_id, article_title, *variables]
    
    # Append to output data
    output_data.append(output_row)

# Create DataFrame for output data
output_columns = ['URL_ID', 'Article_Title', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE',
                  'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE',
                  'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']
output_df = pd.DataFrame(output_data, columns=output_columns)

# Save output to Excel
output_df.to_excel('/kaggle/working/Output.xlsx', index=False)